In [1]:
import numpy as np
import random
import math
import time
import scipy
from scipy import ndimage, misc
import matplotlib.pyplot as plt

In [2]:
im = scipy.ndimage.imread('cat.jpg', flatten=True)
print im.shape, type(im)
a = im.shape[0]
b= im.shape[1]
cat = scipy.misc.imresize(im, (a/40,b/40), interp='bilinear', mode=None)
cat = 1.0 - cat/255.0
print cat.shape

(1720, 2584) <type 'numpy.ndarray'>
(43, 64)


In [274]:
test = np.empty((12*12))
for i in range(144):
    test[i] = i
test = test.reshape((3,12,4))
# print test

In [154]:
DEPTH = 3
STRIDE = 1
# to ensure that the input and output volumes are the same: use P=(F-1)/2 given stride 1.
PADDING = 0
FILTER_SIZE = 2

class ToyNet(object):

    def __init__(self, sizes):
        self.sizes = sizes
        # initialize a list of filters
        self.weights = []
        for i in range(DEPTH):
            self.weights.append([np.random.randn(FILTER_SIZE, FILTER_SIZE)])
        self.biases = np.random.rand(DEPTH,1)
        self.activations = []

    def convolve(self, input_neurons):
        '''
        Assume input image to be of channel one!
        '''
        output_dim1 = (input_neurons.shape[0] - FILTER_SIZE + 2*PADDING)/STRIDE + 1        # num of rows
        output_dim2 =  (input_neurons.shape[1] - FILTER_SIZE + 2*PADDING)/STRIDE + 1       # num of cols

        for i in range(DEPTH):
            self.activations.append(np.empty((output_dim1 * output_dim2)))

        print 'shape of input (rows,cols): ', input_neurons.shape
        print 'shape of output (rows, cols): ','(', output_dim1,',', output_dim2, ')'

        for j in range(DEPTH):
            slide = 0
            row = 0
            print self.activations[j].shape[0]    # one dimensional

            for i in range(self.activations[j].shape[0]):  # loop til the output array is filled up -> one dimensional (600)

                # ACTIVATIONS -> loop through each 2x2 block horizontally
                self.activations[j][i] = sigmoid(np.sum(input_neurons[row:FILTER_SIZE+row, slide:FILTER_SIZE + slide] * self.weights[j][0]) + self.biases[j])
                slide += STRIDE

                if (FILTER_SIZE + slide)-STRIDE >= input_neurons.shape[1]:    # wrap indeces at the end of each row
                    slide = 0
                    row += STRIDE

            self.activations[j] = self.activations[j].reshape((output_dim1, output_dim2))
#         print self.activations


In [155]:
class PoolingLayer(object):

    def __init__(self, width_in, height_in, depth, poolsize = (2,2)):
        '''
        width_in and height_in are the dimensions of the input image
        poolsize is treated as a tuple of filter and stride -> it should work with overlapping pooling
        '''
        self.width_in = width_in
        self.height_in = height_in
        self.depth = depth
        self.poolsize = poolsize
        self.width_out = (self.width_in - self.poolsize[0])/self.poolsize[1] + 1      # num of output neurons
        self.height_out = (self.height_in - self.poolsize[0])/self.poolsize[1] + 1
        print self.height_out, self.width_out

        # initialize empty output matrix
        self.output = np.empty((self.depth, self.width_out * self.height_out))
        self.max_indeces = np.empty((self.depth, self.width_out * self.height_out, 2))
        print 'Shape of pooling layer: ', self.output.shape

    def pool(self, input_image):
        k = 0
        
        # for each filter map
        for j in range(self.depth):
            row = 0
            slide = 0
            for i in range(self.width_out * self.height_out):
                toPool = input_image[j][row:self.poolsize[0] + row, slide:self.poolsize[0] + slide]

                self.output[j][k] = np.amax(toPool)                # calculate the max activation
                index = zip(*np.where(np.max(toPool) == toPool))           # save the index of the max
                if len(index) > 1:
                    index = [index[0]]
                index = index[0][0]+ row, index[0][1] + slide
                self.max_indeces[j][k] = index

                slide += self.poolsize[1]

                # modify this if stride != filter for poolsize 
                if slide >= self.width_in:
                    slide = 0
                    row += self.poolsize[1]
                k += 1
#                 print 'matrix: ', toPool,'max', self.output[j][k-1]
#                 print 'index: ', self.max_indeces[j][k-1]
#                 if k > 10:
#                     break
                
        self.output = self.output.reshape((self.depth, self.height_out, self.width_out))
        self.max_indeces = self.max_indeces.reshape((self.depth, self.height_out, self.width_out, 2))
#         print 'AFTER RESHPAING:', self.output

In [234]:
# class FullyConnectedLayer(object):

#     def __init__(self, width_in, height_in, depth, num_neurons, num_classes):
#         self.width_in = width_in
#         self.height_in = height_in
#         self.depth = depth                
#         self.num_neurons = num_neurons
#         self.num_classes = num_classes
                
#         self.weights = np.random.randn(self.num_neurons, self.depth, self.width_in, self.height_in)
#         self.biases = np.random.randn(self.num_neurons)
        
#         self.output = np.empty((num_neurons, 1))

#     def feedforward(self, a):
#         print len(zip(self.biases, self.weights))
#         i = 0
#         for w,b in zip(self.weights, self.biases):     # num of iterations should be == num of num_neurons
#             # TODO
#             # take dot product from all neurons over full depth and convert into single neuron on FC layer
#             print i, w.shape, a.shape
#             i += 1
#             print (np.inner(w, a) + b).shape
#         return self.output
    
#     def classify(self):
#         pass
#         # I. initialize weights and biases!
#         # II. forward prop
#         # III. Calculate loss
#         # Gradient descent -> backprop + update
            
             

In [263]:
class FullyConnectedLayer(object):

    def __init__(self, width_in, height_in, depth, num_classes):
        self.width_in = width_in
        self.height_in = height_in
        self.depth = depth                
        self.num_neurons = self.width_in * self.height_in
        self.num_classes = num_classes
                
        self.weights = np.random.randn(self.num_neurons, self.depth, self.width_in, self.height_in)
        self.biases = np.random.randn(self.num_neurons)
        
        self.output = np.empty((self.width_in, self.height_in))

    def feedforward(self, a):
        print len(zip(self.biases, self.weights))
#         for w,b in zip(self.weights, self.biases):     # num of iterations should be == num of num_neurons
        # TODO
        # take dot product from all neurons over full depth and convert into single neuron on FC layer
        # for each output neuron -> matrix mul: weight matrix over filters x input image + b
        for j in range(self.width_in):
            for i in range(self.height_in):
                print a
                print a[:][0]
                break
            break
                
            
            
#             print i, w.shape, a.shape
#             i += 1
#             print (np.inner(w, a) + b).shape
#         return self.output
    
    def classify(self):
        pass
        # I. initialize weights and biases!
        # II. forward prop
        # III. Calculate loss
        # Gradient descent -> backprop + update
            
             

In [264]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

In [ ]:
# net = ToyNet([test.shape[0]*test.shape[1]])
# print 'yooooo', net.sizes[0]
# net.convolve(test)

# TODO: implement for all activations!
# pool_layer = PoolingLayer(12, 12, 1) # only implemented for the first depth layer
# pool_layer.pool(test)
fc = FullyConnectedLayer(3,12,4)

In [291]:
test = np.empty((12*12))
for i in range(144):
    test[i] = i
test = test.reshape((3,12,4))
# print test

[[ 0.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]]
